In [1]:
import requests
import json
import re
import pandas as pd

In this code we are using YAP model (developed by ONLP Lab)
https://github.com/OnlpLab/yap

# Functions

In [2]:
noun = ['NN', 'NNG', 'NNGT', 'NNP', 'NNT']
verb = ['RB', 'RBR', 'VB']

In [3]:
def gender(st):
    gender = re.split("gen=", st)
    try:
        if len(gender)>1:
            return str(gender[1][0])
        else:
            return
    except:
        return

In [4]:
def find_gender(df):
    gender = 'M'
    for i in range(len(df)):
#         if df.loc[i,4] in noun and df.loc[i,'gender']=='F':
#             gender = 'F'
        if df.loc[i,4] in verb and df.loc[i,'gender']=='F':
            return 'F'
    return gender

In [11]:
bag_of_words = []
filepath = 'bag_of_words.txt'
with open(filepath) as fp:
    line = fp.readline()
    bag_of_words.append(line.strip())
    cnt = 1
    while line:
        line = fp.readline()
        bag_of_words.append(line.strip())
        cnt += 1

In [12]:
for word in bag_of_words:
    word.strip()

In [13]:
bag_of_words.remove("")

In [14]:
def find_words(list_of_words):
    words_found = []
    for word in list_of_words:
        if word in bag_of_words:
            words_found.append(word)
    return words_found

In [15]:
def find_subject_and_words(r):
    result = r.content
    text = result.decode('utf-8')
    d = json.loads(text)
    txt = d['md_lattice']
    tokens = re.split("\t", txt)
    df = []
    line = []

    #creating dataframe from string
    for t in tokens:
        if '\n' in t:
            line.append(re.split("\n", t)[0])
            df.append(line)
            line = [] 
            line.append(re.split("\n", t)[1])
        else:
            line.append(t)
    df = pd.DataFrame(df)
#     df = df.drop([0,1,3,5], axis=1)
    df = df.rename(columns={6: "gender"})
    #finding gender
    df['gender'] = df['gender'].apply(gender)
    gender_subject = find_gender(df)

    #finding the words from bag of words
    bad_words = find_words(df[2])
    
    return gender_subject, bad_words

# Read JSON

In [16]:
with open('../data1/data1.json', 'r') as myfile:
    data = myfile.read()
    posts = json.loads(data)

In [17]:
posts = posts['data']

# NLP (Using YAP)

In [19]:
data2 = []

In [20]:
URL = "http://localhost:8000/yap/heb/joint"
for post in posts:
    try:
        post_data2 = {
            "source": post["source"],
            "header": post["header"].strip(),
            "sub_header": post["sub_header"].strip(),
            "img": post["img"]
        }

        post_content = post['header']+"  "   
        PARAMS = {"text": post_content}
        r = requests.get(url = URL, json = PARAMS) 
        res_title = find_subject_and_words(r)
        post_content = post['sub_header']+"  "   
        PARAMS = {"text": post_content}
        r = requests.get(url = URL, json = PARAMS) 
        res_sub_title = find_subject_and_words(r)
        if res_title[0] == 'M':
            if res_sub_title[0] == 'M':
                post_data2["gender"] = "M"
                post_data2["flag"] = '0'
            else:
                post_data2["gender"] = "F"
                post_data2["words"] = res_sub_title[1]
                if len(res_sub_title[1])>0:
                    post_data2["flag"] = '1'
                else:
                    post_data2["flag"] = '0'
        else:
            post_data2["gender"] = "F"
            if res_sub_title[0] == 'M':
                post_data2["words"] = res_title[1]
                if len(res_title[1])>0:
                    post_data2["flag"] = '1'
                else:
                    post_data2["flag"] = '0'
            else:
                post_data2["words"] = res_title[1]+res_sub_title[1]
                if len(res_title[1])+len(res_sub_title[1])>0:
                    post_data2["flag"] = '1'
                else:
                    post_data2["flag"] = '0'
        data2.append(post_data2)          
    except:
        continue
            

# Export

In [25]:
data2 = {"data":data2}

In [26]:
with open("data2.json","w", encoding='utf-8') as jsonfile:
        json.dump(data2,jsonfile,ensure_ascii=False)

In [ ]:
# definitions = {
#         "CC": "Coordinating conjunction",
#         "CD": "Cardinal number",
#         "EX": "Determiner",
#         "FW": "Foreign word",
#         "IN": "Preposition",
#         "JJ": "Adjective",
#         "VP": "Verb",

#         "JJR": "Adjective",
#         "JJS": "Adjective",
#         "LS": "List",
#         "MD": "Modal",

#         "NN": "Noun",
#         "NNS": "Noun",
#         "PP": "Preposition",
#         "NNP": "Noun",
#         "NNPS": "Pre determiner",
#         "PDT": "Verb",
#         "TO": "to",
#         "POS": "Possessive ending",
#         "PRP": "Personal pronoun",
#         "PRP$": "Personal pronouns",
#         "RB": "Adverb",
#         "RBR": "Adverb",
#         "RP": "Particle",
#         "S": "Simple declarative",
#         "SBAR": "Clause introduced",
#         "SBARQ": "null",
#         "SINV": "null",
#         "DT": "determiner",
#         "SQ": "null",
#         "SYM": "Symbol",
#         "VB": "Verb",
#         "UH": "interjection",
#         "VBD": "Verb",
#         "VBG": "Verb",
#         "VBN": "Verb",
#         "VBP": "Verb",
#         "VBZ": "Verb",
#         "WP": "Pronoun",
#         "WRB": "Adverb",
#         ":": "none",
#         "": "none",
#         "WDT": "Wh-determiner"
#     }